In [3]:
import pandas as pd
from tqdm import tqdm

# Chemin vers le fichier CSV d'entrée et de sortie
input_file = "StockEtablissement_utf8.csv"
output_file = "siren_naf_45.csv"

# Taille des morceaux (ajustez selon votre mémoire disponible)
chunk_size = 100000  # 100 000 lignes par morceau

# Liste pour stocker les morceaux filtrés
chunks_filtres = []

# Colonnes à conserver
colonnes_a_garder = [
    "siren", "siret", "dateCreationEtablissement",
    "activitePrincipaleRegistreMetiersEtablissement",
    "dateDernierTraitementEtablissement", "dateDebut",
    "activitePrincipaleEtablissement", "etatAdministratifEtablissement",
    "denominationUsuelleEtablissement", "complementAdresseEtablissement", "complementAdresse2Etablissement",
    "codeCommuneEtablissement", "codePostalEtablissement",
    "trancheEffectifsEtablissement",
    "enseigne1Etablissement", "enseigne2Etablissement", "enseigne3Etablissement"
]

# Départements des Hauts-de-France
departements_hdf = ['59', '62', '80', '60', '02']

# Lire le fichier par morceaux avec une barre de progression
reader = pd.read_csv(input_file, sep=",", chunksize=chunk_size, encoding="utf-8", usecols=colonnes_a_garder)

for chunk in tqdm(reader, desc="Traitement des morceaux"):
    # Filtrer les lignes où l'une des colonnes commence par "45", l'établissement est actif ("A"),
    # et où le code commune ou postal commence par un des départements HDF
    masque_activite = (
        (chunk["activitePrincipaleRegistreMetiersEtablissement"].astype(str).str.startswith("45")) |
        (chunk["activitePrincipaleEtablissement"].astype(str).str.startswith("45"))
    )

    masque_etat = (chunk["etatAdministratifEtablissement"] == "A")

    masque_hdf = (
        chunk["codeCommuneEtablissement"].astype(str).str.startswith(tuple(departements_hdf)) |
        chunk["codePostalEtablissement"].astype(str).str.startswith(tuple(departements_hdf))
    )

    masque = masque_activite & masque_etat & masque_hdf

    chunk_filtre = chunk[masque]

    # Ajouter le morceau filtré à la liste
    if not chunk_filtre.empty:
        chunks_filtres.append(chunk_filtre)

# Concaténer tous les morceaux filtrés en un seul DataFrame
df_filtre = pd.concat(chunks_filtres, ignore_index=True)

# Afficher un aperçu du DataFrame résultant
print(f"Nombre total de lignes filtrées (établissements actifs dans les HDF) : {len(df_filtre)}")

# Enregistrer le DataFrame filtré dans un fichier CSV avec ";" comme séparateur
df_filtre.to_csv(output_file, index=False, encoding="utf-8", sep=";")

print(f"Fichier enregistré sous : {output_file}")


Traitement des morceaux: 0it [00:00, ?it/s]

c:\Python310\lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (7,22) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Traitement des morceaux: 1it [00:00,  1.59it/s]c:\Python310\lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Traitement des morceaux: 2it [00:01,  1.71it/s]c:\Python310\lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (7,18,22,30,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Traitement des morceaux: 3it [00:01,  1.67it/s]c:\Python310\lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (18,22,30,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Traitement des morceaux: 4it [00:02,  1.58it/s]c:\Python310\lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (22,47,49) have mi

Nombre total de lignes filtrées (établissements actifs dans les HDF) : 19028
Fichier enregistré sous : siren_naf_45.csv


In [ ]:
df_filtre